# ModRev tutorial

This notebook illustrates the main features of the python API of the `ModRev` toolkit. It will show how to load a model and perform various operations on it.

The code from this tutorial can be executed on the standard python interpreter, the IPython shell or the Jupyter notebook web interface.

In [ ]:
import biolqm
import pymodrev

In [ ]:
lqm = biolqm.load("/opt/ModRev/examples/model.lp")

Note:
Example from CSMB2020 paper is ->
modrev -m /opt/ModRev/examples/model.lp -obs /opt/ModRev/examples/obsTS01.lp -up s -v 0

ModRev models can be created from lqm models

In [ ]:
model = pymodrev.ModRev(lqm)

We can print the model to check how it is represented in the ModRev file format

In [ ]:
model.print() 

First of all, we can check the consistency of the model

In [ ]:
model.is_consistent()

**Observations**

Observations can be added to the model, and will always be considered when checking consistency, generating repairs, etc...

To add observations we can pass a list of values or a dictionary with the node names as keys.

For example:

Note: by default, if node is not present in observation, it is considered as missing value

In [ ]:
model.add_obs([0, "*", 1], "obs_1")

In [ ]:
model.add_obs({"v1": 0, "v2": "*", "v3": 1})

We can also set the observation dictionary by hand all at once, substituting the previous observations

In [ ]:
model.set_obs({"obs_1": [0, 1, "*"],
               "obs_2": {"v1": 0, "v2": 1, "v3": "*"}
              })

We can see the current observations:

In [ ]:
model.get_observations()

And finally, we can remove them:

In [ ]:
model.remove_obs("obs_1")

The following function can be used to check the possible repair actions, based on the model and loaded observations.

In [ ]:
model.stats()

Additionally, we can also check the stats with observations from a file, instead of our observation dictionary.

To do this, we also need to specify if these are steady state or time-series (synchronous) observations.

In [ ]:
model.stats(observation_file="/opt/ModRev/examples/obsSS.lp", state_scheme="steady")

In [ ]:
model.stats(observation_file="/opt/ModRev/examples/obsTS01.lp", state_scheme="synchronous")

From the list of possible repairs for each inconsistent node, we can specify in a dictionary which ones we want to pick.
For example, for this given inconsistent network:

Repair options for node 'v1':
- v1@F1, (v2) || (v3) -----> repair 0

Repair options for node 'v2':
- E,v1,v2:F,(v1 && v3) -------> repair 0
- E,v3,v2:F,(v1 && v3) -------> repair 1

Repair options for node 'v3':
- v3@F1, (v1) || (v2) -----> repair 0
- v3@F2, (v1) || (v2) -----> repair 1

We could specify the following repairs:
```python
model.generate_repairs({'v1': 0, 'v2': 1, 'v3': 0})
```

Additionally, if we want to pass a list of fixed nodes, we can do it as follows:
model.generate_repairs({'v1': 0, 'v2': 1, 'v3': 0}, fixed_nodes=['v1', 'v3'])

And modrev will try to repair the network while keeping the nodes 'v1' and 'v3' fixed.


In [ ]:
new_model = model.generate_repairs({'v1': 0, 'v2': 0, 'v3': 0}, fixed_nodes=[])

And we can check our new model

In [ ]:
new_model.print()

### **Example of a remote file**

The following code has intentional errors (incorrect arguments, usage, etc.), for demonstration purposes.

In [ ]:
lqm = biolqm.load("http://ginsim.org/sites/default/files/boolean_cell_cycle.zginml")

In [ ]:
model = pymodrev.ModRev(lqm)

In [ ]:
model.print()

In [ ]:
model.stats()

In [ ]:
model.add_obs([0, 1, "*", 1, 0], "obs_1")

In [ ]:
model.add_obs({"CycA": 0, "node2": 1, "CycD": "*", "CycB": 1, "CycE": 0})

In [ ]:
model.set_obs({"obs_1": [0, 1, "*", 1, 0],
               "obs_2": {"CycA": 0, "node2": 1, "CycB": "*", "CycE": 1, "p27": 0}
              })

In [ ]:
model.repairs['p27'] = ["p27@F1, (CycA) || (CycB) || (CycE)"]

In [ ]:
new_model = model.generate_repairs({'p27': 0}, fixed_nodes=['CycA', 'CycB', 'p27'])

In [ ]:
new_model.print()